Nicolas Ramos Samboni - Big Data 2021/2022
Evaluación por pares Módulo 3: “Big Data y Geospatial Data Science”

In [12]:
#import the needed pckg
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt

In [41]:
satelites_data = pd.read_csv('satelliteDB.csv')
satelites_data.head(3)

,Official Name of Satellite,Country/Organization of UN Registry,Operator/Owner,Country of Operator/Owner,Users,Purpose,Detailed Purpose,Class of Orbit,Type of Orbit,Longitude of Geosynchronous Orbit (Degrees),...,Dry Mass (Kilograms),Power (Watts),Date of Launch,Expected Lifetime (Years),Contractor,Country of Contractor,Launch Site,Launch Vehicle,COSPAR Number,NORAD Number
0,AAUSat-4,NR,University of Aalborg,Denmark,Civil,Earth Observation,Automatic Identification System (AIS),LEO,Sun-Synchronous,0.0,...,NaN,NaN,4/25/2016,NaN,University of Aalborg,Denmark,Guiana Space Center,Soyuz 2.1a,2016-025E,41460.0
1,ABS-2,NR,Asia Broadcast Satellite Ltd.,Multinational,Commercial,Communications,NaN,GEO,NaN,75.0,...,NaN,16000,2/6/2014,15,Space Systems/Loral,USA,Guiana Space Center,Ariane 5 ECA,2014-006A,39508.0
2,ABS-2A,NR,Asia Broadcast Satellite Ltd.,Multinational,Commercial,Communications,NaN,GEO,NaN,-75.0,...,NaN,NaN,6/15/2016,15,Boeing Satellite Systems,USA,Cape Canaveral,Falcon 9,2016-038A,41588.0


Glossary:
    Class of Orbit: Depending on what the satellite is designed to achieve. There are many classes of orbits (geostatinary, low earth orbit...)
    Type of Orbit: 
    Longitude of Geosynchronous Orbit:
    Perigee:
    Apogee:
    Eccentricity:
    Inclination:
    Period: 

In [40]:
for column in satelites_data.columns:
    print(str(type(column)) + column)
#we'll need to convert the data type for numerical operations and some plots

<class 'str'>Official Name of Satellite
<class 'str'>Country/Organization of UN Registry
<class 'str'>Operator/Owner
<class 'str'>Country of Operator/Owner
<class 'str'>Users
<class 'str'>Purpose
<class 'str'>Detailed Purpose
<class 'str'>Class of Orbit
<class 'str'>Type of Orbit
<class 'str'>Longitude of Geosynchronous Orbit (Degrees)
<class 'str'>Perigee (Kilometers)
<class 'str'>Apogee (Kilometers)
<class 'str'>Eccentricity
<class 'str'>Inclination (Degrees)
<class 'str'>Period (Minutes)
<class 'str'>Launch Mass (Kilograms)
<class 'str'>Dry Mass (Kilograms)
<class 'str'>Power (Watts)
<class 'str'>Date of Launch
<class 'str'>Expected Lifetime (Years)
<class 'str'>Contractor
<class 'str'>Country of Contractor
<class 'str'>Launch Site
<class 'str'>Launch Vehicle
<class 'str'>COSPAR Number
<class 'str'>NORAD Number
